# All needed imports

In [3]:
import os

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.layers import Input, Conv2D, Reshape, UpSampling2D, MaxPool2D, Dense, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\user\desktop\diploma\autoencoder\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "c:\users\user\desktop\diploma\autoencoder\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "c:\users\user\desktop\diploma\autoencoder\venv\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\User\AppData\Local\Programs\Python\Python37\lib\imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\User\AppData\Local\Programs\Python\Python37\lib\imp.py", line 343, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Не найден указан

TypeError: can only concatenate str (not "list") to str

In [ ]:
sess = tf.Session()
K.set_session(sess)

# Load mnist dataset

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test .astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test,  (len(x_test),  28, 28, 1))

y_train_cat = to_categorical(y_train).astype(np.float32)
y_test_cat  = to_categorical(y_test).astype(np.float32)

## Create AE submodels 

In [ ]:
batch_size = 500
latent_dim = 2
dropout_rate = 0.3

### Encoder

In [ ]:
input_ae_encoder = Input(shape=(28, 28, 1))

x = Conv2D(32, kernel_size=(5, 5))(input_ae_encoder)
x = LeakyReLU()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Conv2D(16, kernel_size=(5, 5))(x)
x = LeakyReLU()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

begin_of_dense = Flatten()(x)
x = LeakyReLU()(begin_of_dense)
x = Dense(latent_dim * 4)(x)
x = LeakyReLU()(x)
encoded = Dense(latent_dim, activation='relu')(x)
ae_encoder = Model(input_ae_encoder, encoded, name='Encoder')

### Decoder

In [ ]:
input_ae_decoder = Input(shape=(latent_dim,))

x = Dense(49)(input_ae_decoder)
x = Reshape(target_shape=(7, 7, 1))(x)
x = Conv2D(16, kernel_size=(7, 7), padding='same')(x)
x = LeakyReLU()(x)
x = UpSampling2D(size=(2, 2))(x)
x = Conv2D(32, kernel_size=(5, 5), padding='same')(x)
x = LeakyReLU()(x)
x = UpSampling2D(size=(2, 2))(x)
decoded = Conv2D(1, kernel_size=(7, 7), padding='same', activation='sigmoid')(x)
ae_decoder = Model(input_ae_decoder, decoded, name='Decoder')

### AE Model

In [ ]:
ae_model = Model(input_ae_encoder, ae_decoder(ae_encoder(input_ae_encoder)), name='ConvAE')
ae_model.compile(loss='binary_crossentropy', optimizer='adam')
ae_model.summary()

### AE Model Training

In [ ]:
import time

from keras.callbacks import LambdaCallback, TensorBoard


imgs = x_test[:batch_size]
# imgs_lbls = y_test_cat[:batch_size]
n_compare = 10

epochs_to_check = np.linspace(0, 100, num=10).astype('int')


digit_size = 28
def plot_digits(*args):
    args = [x.squeeze() for x in args]
    n = min([x.shape[0] for x in args])
    figure = np.zeros((digit_size * len(args), digit_size * n))

    for i in range(n):
        for j in range(len(args)):
            figure[j * digit_size: (j + 1) * digit_size,
                   i * digit_size: (i + 1) * digit_size] = args[j][i].squeeze()

    plt.figure(figsize=(2 * n, 2 * len(args)))
    plt.imshow(figure, cmap='Greys_r')
    plt.grid(False)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.show()


def on_epoch_end(epoch, logs):
    if epoch in epochs_to_check:
        decoded_imgs = ae_model.predict(imgs, batch_size=batch_size)
        plot_digits(imgs[:n_compare], decoded_imgs[:n_compare])


plt_fig = LambdaCallback(on_epoch_end=on_epoch_end)
tb      = TensorBoard(log_dir='./logs')

start_time = time.time()
ae_model.fit(x_train, x_train,
              batch_size=batch_size,
              epochs=200,
              shuffle=True,
              validation_data=(x_test, x_test),
              callbacks=[plt_fig, tb],
              verbose=1)
print("ALL AE TRAINING ELAPSED TIME: %s" % (time.time() - start_time))

### Try to decode random codes

In [ ]:
codes = np.random.randn(n_compare, latent_dim)
decoded_from_codes = ae_decoder.predict(codes, batch_size=n_compare)
plot_digits(decoded_from_codes)


# SAVE MODELS

In [ ]:
import os
import json

save_path = os.path.join("saved_models", "convolutional_ae")
if not os.path.exists(save_path):
    os.mkdir(save_path)

with open(os.path.join(save_path, "ae_2_ldim_200_epochs.json"), "w") as f:
    json.dump(ae_model.to_json(), f, indent=4)
ae_model.save_weights(os.path.join(save_path, "ae.h5"))

with open(os.path.join(save_path, "ae_encoder_2_ldim_200_epochs.json"), "w") as f:
    json.dump(ae_encoder.to_json(), f, indent=4)
ae_encoder.save_weights(os.path.join(save_path, "ae_encoder_2_ldim_200_epochs.h5"))

with open(os.path.join(save_path, "ae_decoder_2_ldim_200_epochs.json"), "w") as f:
    json.dump(ae_decoder.to_json(), f, indent=4)
ae_decoder.save_weights(os.path.join(save_path, "ae_decoder_2_ldim_200_epochs.h5"))

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir ./logs